In [1]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd
import fiona

from ipyleaflet import Map, GeoJSON, projections, basemaps, GeoData, LayersControl, WidgetControl, GeoJSON
from ipywidgets import Text, HTML

/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/google/auth/_default.py:68: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
census_pop = pd.read_csv('./ACSDT5Y2019.B01003_2021-10-18T202433/ACSDT5Y2019.B01003_data_with_overlays_2021-10-18T202425.csv')
census_pop = census_pop[1:]
census_pop = census_pop.rename(columns={'B01003_001E':'population', 'B01003_001M':'margin_err'})
census_pop['GEOID'] = census_pop['GEO_ID'].apply(lambda x: x[9:])
census_pop['population'] = census_pop['population'].astype('int64')

In [3]:
census_pop

,GEO_ID,NAME,population,margin_err,GEOID
1,1400000US06001400100,"Census Tract 4001, Alameda County, California",3120,208,06001400100
2,1400000US06001400200,"Census Tract 4002, Alameda County, California",2007,120,06001400200
3,1400000US06001400300,"Census Tract 4003, Alameda County, California",5051,309,06001400300
4,1400000US06001400400,"Census Tract 4004, Alameda County, California",4007,344,06001400400
5,1400000US06001400500,"Census Tract 4005, Alameda County, California",4124,357,06001400500
...,...,...,...,...,...
8053,1400000US06115040800,"Census Tract 408, Yuba County, California",4652,247,06115040800
8054,1400000US06115040901,"Census Tract 409.01, Yuba County, California",2720,298,06115040901
8055,1400000US06115040902,"Census Tract 409.02, Yuba County, California",1653,334,06115040902
8056,1400000US06115041000,"Census Tract 410, Yuba County, California",7298,789,06115041000


In [4]:
census_shape = gpd.read_file('./tl_2019_06_tract/tl_2019_06_tract.shp')

In [5]:
census_joined = census_shape >> inner_join(_, census_pop, on='GEOID')

In [6]:
census_joined = census_joined.to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)

In [7]:
sq_mi_per_sq_m = 3.86 * 10**-7

In [8]:
census_joined['sq_mi'] = census_joined['geometry'].area * sq_mi_per_sq_m

In [9]:
census_joined['urban'] = census_joined.apply(lambda x: True if x.population / x.sq_mi > 2400 else False, axis=1)

In [10]:
bus_hqta = gpd.read_parquet('../high_quality_transit_areas/data/bus/shape_hqta_dissolve.parquet')

In [17]:
def map_hqta(gdf, gdf2, mouseover=None):
    global nix_list
    nix_list = []
    
    if 'calitp_extracted_at' in gdf.columns:
        gdf = gdf.drop(columns='calitp_extracted_at')
    gdf = gdf.to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)
    if gdf.geometry.iloc[0].geom_type == 'Point':
        gdf.geometry = gdf.geometry.buffer(200)
    
    x = gdf.to_crs('EPSG:4326').geometry.iloc[0].centroid.x
    y = gdf.to_crs('EPSG:4326').geometry.iloc[0].centroid.y
    
    m = Map(basemap=basemaps.CartoDB.Positron, center=[y, x], zoom=11)

    if mouseover:
        html = HTML(f'hover to see {mouseover}')
        html.layout.margin = '0px 20px 20px 20px'
        control = WidgetControl(widget=html, position='topright')
        m.add_control(control)

        def update_html(feature,  **kwargs):
            html.value = '''
                <h3><b>{}</b></h3>
            '''.format(feature['properties'][mouseover])
            
        def add_to_nix(feature, **kwargs):
            nix_list.append(feature['properties'][mouseover])
            
    if 'urban' in gdf.columns:
        geo_data_urban = GeoData(geo_dataframe = gdf[gdf['urban']].to_crs('EPSG:4326'),
                               style={'color': 'black', 'fillColor': '#2b83ba',
                                            'opacity':0.4, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                               hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                               name = 'HQTA')
        #a8ddb5
        geo_data_not_urban = GeoData(geo_dataframe = gdf[~gdf['urban']].to_crs('EPSG:4326'),
                               style={'color': 'black', 'fillColor': '#abdda4',
                                            'opacity':0.2, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                               hover_style={'fillColor': 'red' , 'fillOpacity': 0.1},
                               name = 'non-HQTA')

        m.add_layer(geo_data_urban)
        m.add_layer(geo_data_not_urban)
    
    if 'hq_transit_corr' in gdf2.columns:
        
        geo_data_hq = GeoData(geo_dataframe = gdf2[gdf2['hq_transit_corr']].to_crs('EPSG:4326'),
                               style={'color': 'black', 'fillColor': '#d7191c',
                                            'opacity':0.2, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                               hover_style={'fillColor': 'red' , 'fillOpacity': 0.3},
                               name = 'HQTA')
        #a8ddb5
        geo_data_not_hq = GeoData(geo_dataframe = gdf2[~gdf2['hq_transit_corr']].to_crs('EPSG:4326'),
                               style={'color': 'black', 'fillColor': '#fdae61',
                                            'opacity':0.2, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                               hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                               name = 'non-HQTA')

        m.add_layer(geo_data_hq)
        m.add_layer(geo_data_not_hq)
        
        legend = LegendControl({"Urban tract":"#2b83ba", "Non-urban tract":"#abdda4", "HQ transit corridor":"#d7191c",
                               "Other transit corridor":"fdae61"},
                               name="Legend",
                               position="bottomright")
        m.add_control(legend)
        
    if mouseover:
        geo_data_hq.on_hover(update_html)
        geo_data_hq.on_hover(add_to_nix)

    m.add_control(LayersControl())

    return m

In [14]:
map_hqta(census_joined, bus_hqta)

Map(center=[34.178153601202446, -118.55812675253787], controls=(ZoomControl(options=['position', 'zoom_in_text…